In [ ]:
######

#Firstly create a tensor which contains the iamges and their labels

######

In [53]:
import os
import torch
from torchvision import transforms
from PIL import Image

# Define the folder containing the images
image_folder = '/Users/louieburns/Library/CloudStorage/OneDrive-UniversityofLeeds/Year 3/AI and Machine Learning/Term 1/Coursework 1/Actual Coursework/dataoriginal/images'

# Path to the text file
label_file = '/Users/louieburns/Library/CloudStorage/OneDrive-UniversityofLeeds/Year 3/AI and Machine Learning/Term 1/Coursework 1/Actual Coursework/dataoriginal/images_family_train.txt'

# Define transformations for image processing
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to 128x128 (can be adjusted)
    transforms.ToTensor()          # Convert to tensor
])

# Parse the label file and create a mapping from unique labels to integers starting at 0
label_mapping = {}
image_data = []
labels = []

with open(label_file, 'r') as f:
    for line in f:
        parts = line.strip().split(maxsplit=1)
        if len(parts) != 2:
            continue  # Skip malformed lines
        filename, label = parts
        
        # Assign a new integer label if not already mapped
        if label not in label_mapping:
            label_mapping[label] = len(label_mapping)
        
        # Process the image
        image_path = os.path.join(image_folder, filename + ".jpg")
        if os.path.exists(image_path):
            try:
                image = Image.open(image_path).convert('RGB')
                image_tensor = transform(image)
                image_data.append(image_tensor)
                labels.append(label_mapping[label])
            except Exception as e:
                print(f"Error processing {image_path}: {e}")

# Convert lists to PyTorch tensors
image_tensor = torch.stack(image_data)  # Create a tensor from image data
label_tensor = torch.tensor(labels)     # Convert labels to tensor

# Print label mapping and tensor dimensions for verification
print("Label Mapping:", label_mapping)
print("Image tensor shape:", image_tensor.shape)
print("Label tensor shape:", label_tensor.shape)



Label Mapping: {'Boeing 707': 0, 'Boeing 727': 1, 'Boeing 737': 2, 'Boeing 747': 3, 'Boeing 757': 4, 'Boeing 767': 5, 'Boeing 777': 6, 'A300': 7, 'A310': 8, 'A320': 9, 'A330': 10, 'A340': 11, 'A380': 12, 'ATR-42': 13, 'ATR-72': 14, 'An-12': 15, 'BAE 146': 16, 'BAE-125': 17, 'Beechcraft 1900': 18, 'Boeing 717': 19, 'C-130': 20, 'C-47': 21, 'CRJ-200': 22, 'CRJ-700': 23, 'Cessna 172': 24, 'Cessna 208': 25, 'Cessna Citation': 26, 'Challenger 600': 27, 'DC-10': 28, 'DC-3': 29, 'DC-6': 30, 'DC-8': 31, 'DC-9': 32, 'DH-82': 33, 'DHC-1': 34, 'DHC-6': 35, 'Dash 8': 36, 'DR-400': 37, 'Dornier 328': 38, 'Embraer E-Jet': 39, 'EMB-120': 40, 'Embraer ERJ 145': 41, 'Embraer Legacy 600': 42, 'Eurofighter Typhoon': 43, 'F-16': 44, 'F/A-18': 45, 'Falcon 2000': 46, 'Falcon 900': 47, 'Fokker 100': 48, 'Fokker 50': 49, 'Fokker 70': 50, 'Global Express': 51, 'Gulfstream': 52, 'Hawk T1': 53, 'Il-76': 54, 'L-1011': 55, 'MD-11': 56, 'MD-80': 57, 'MD-90': 58, 'Metroliner': 59, 'King Air': 60, 'PA-28': 61, 'SR-20

In [19]:
##########

#Create the testing data set

#########

In [55]:
# Path to the testing data file
testing_file = '/Users/louieburns/Library/CloudStorage/OneDrive-UniversityofLeeds/Year 3/AI and Machine Learning/Term 1/Coursework 1/Actual Coursework/dataoriginal/images_family_test.txt'

test_image_data = []
test_labels = []

with open(testing_file, 'r') as f:
    for line in f:
        parts = line.strip().split(maxsplit=1)  # Split into filename and label
        if len(parts) != 2:
            continue  # Skip malformed lines
        filename, label = parts

        # Check if the label exists in the training label mapping
        if label not in label_mapping:
            print(f"Warning: Label '{label}' in testing data not found in training labels.")
            continue  # Skip labels not seen during training

        # Construct image path and process if exists
        image_path = os.path.join(image_folder, filename + ".jpg")  # Assuming .jpg extension
        if os.path.exists(image_path):
            try:
                image = Image.open(image_path).convert('RGB')  # Open image
                image_tensor = transform(image)               # Apply transformations
                test_image_data.append(image_tensor)
                test_labels.append(label_mapping[label])      # Map the label to its integer value
            except Exception as e:
                print(f"Error processing file {image_path}: {e}")
        else:
            print(f"File not found: {image_path}")

# Ensure data synchronization
assert len(test_image_data) == len(test_labels), "Mismatch in number of test images and labels!"

# Convert to tensors
test_image_tensor = torch.stack(test_image_data)  # Stack images into a tensor
test_label_tensor = torch.tensor(test_labels)     # Convert labels to a tensor

# Verify shapes
print("Test Image Tensor Shape:", test_image_tensor.shape)
print("Test Label Tensor Shape:", test_label_tensor.shape)


Test Image Tensor Shape: torch.Size([3333, 3, 128, 128])
Test Label Tensor Shape: torch.Size([3333])


In [ ]:
########

#Creating the CNN

########


In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Input channels = 3 (RGB), Output channels = 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling
        self.fc1_input_size = self._get_flatten_size()
        self.fc1 = nn.Linear(self.fc1_input_size, 128)  # Adjusted input size
        self.fc2 = nn.Linear(128, num_classes)

    def _get_flatten_size(self):
        # Use a dummy tensor to calculate the flattened size
        with torch.no_grad():
            dummy_input = torch.zeros(1, 3, 128, 128)  # Example input tensor
            x = self.pool(torch.relu(self.conv1(dummy_input)))
            x = self.pool(torch.relu(self.conv2(x)))
            return x.view(1, -1).size(1)  # Flatten and get the size

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Parameters
num_classes = len(label_mapping)  # Number of unique labels
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Ensure label tensor shape is correct
label_tensor = label_tensor.squeeze()  # Remove extra dimensions if any

image_tensor = torch.stack(image_data)  # Ensure proper tensor stacking for images
label_tensor = torch.tensor(labels, dtype=torch.long)  # Ensure labels are integer tensors

# Create TensorDataset
train_dataset = TensorDataset(image_tensor, label_tensor)
test_dataset = TensorDataset(test_image_tensor, test_label_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNN(num_classes=num_classes)
criterion = nn.CrossEntropyLoss()  # Use cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the CNN
print("Training the CNN...")
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimize weights

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Evaluate the CNN on the test set
print("Evaluating the CNN...")
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Training the CNN...


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x262144 and 65536x128)

In [ ]:
##### 

#F1 Calculation Score

#####

In [ ]:
from sklearn.metrics import f1_score

# Evaluate the CNN on the test set and calculate accuracy and F1-score
print("Evaluating the CNN...")
model.eval()  # Set the model to evaluation mode
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
        all_labels.extend(labels.numpy())      # Store true labels
        all_predictions.extend(predicted.numpy())  # Store predictions

# Calculate Accuracy
correct = sum(1 for true, pred in zip(all_labels, all_predictions) if true == pred)
total = len(all_labels)
accuracy = 100 * correct / total

# Calculate F1-score
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"Test Accuracy: {accuracy:.2f}%")
print(f"F1 Score: {f1:.2f}")
